<a href="https://colab.research.google.com/github/ROHIT251/RESUME_TO_JOB_MATCHER_ROHIT_RAM/blob/main/Resume_to_Job_Matcher_ROHIT_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy transformers sentence-transformers langchain openai fastapi uvicorn gradio nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [ ]:
import spacy
import nltk
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import openai
import torch
import fastapi
import gradio as gr

In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Load sentence-transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
resume_df = pd.read_csv('UpdatedResumeDataSet.csv')
job_df = pd.read_csv('job_title_des.csv')

# Display first few rows
resume_df.head(), job_df.head()

(       Category                                             Resume
 0  Data Science  Skills * Programming Languages: Python (pandas...
 1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...
 2  Data Science  Areas of Interest Deep Learning, Control Syste...
 3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
 4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...,
    Unnamed: 0             Job Title  \
 0           0     Flutter Developer   
 1           1      Django Developer   
 2           2      Machine Learning   
 3           3         iOS Developer   
 4           4  Full Stack Developer   
 
                                      Job Description  
 0  We are looking for hire experts flutter develo...  
 1  PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...  
 2  Data Scientist (Contractor)\n\nBangalore, IN\n...  
 3  JOB DESCRIPTION:\n\nStrong framework outside o...  
 4  job responsibility full stack engineer – react...  )

In [ ]:
def preprocess_text(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

# Apply preprocessing
resume_df["cleaned_resume"] = resume_df["Resume"].apply(preprocess_text)
job_df["cleaned_job"] = job_df["Job Description"].apply(preprocess_text)

# Display processed data
resume_df.head(), job_df.head()

(       Category                                             Resume  \
 0  Data Science  Skills * Programming Languages: Python (pandas...   
 1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...   
 2  Data Science  Areas of Interest Deep Learning, Control Syste...   
 3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...   
 4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...   
 
                                       cleaned_resume  
 0  skill programming language python pandas numpy...  
 1  education detail uit rgpv data scientist datum...  
 2  area interest deep learning control system des...  
 3  skill r python sap hana tableau sap hana sql s...  
 4  education detail mca ymcaust faridabad haryana...  ,
    Unnamed: 0             Job Title  \
 0           0     Flutter Developer   
 1           1      Django Developer   
 2           2      Machine Learning   
 3           3         iOS Developer   
 4           4  Full Stack Develope

In [ ]:
# Encode job descriptions
job_embeddings = model.encode(job_df["cleaned_job"].tolist(), convert_to_tensor=True)

# Encode resumes
resume_embeddings = model.encode(resume_df["cleaned_resume"].tolist(), convert_to_tensor=True)

In [ ]:
# Compute cosine similarity scores
similarity_matrix = util.pytorch_cos_sim(resume_embeddings, job_embeddings)

# Convert to a NumPy array
similarity_scores = similarity_matrix.numpy()

# Display the top match for each resume
top_matches = np.argmax(similarity_scores, axis=1)

for i, job_idx in enumerate(top_matches[:5]):  # Show first 5 matches
    print(f"Resume {i+1} best matches Job {job_idx+1} with similarity {similarity_scores[i, job_idx]:.2f}")

Resume 1 best matches Job 330 with similarity 0.83
Resume 2 best matches Job 682 with similarity 0.81
Resume 3 best matches Job 655 with similarity 0.78
Resume 4 best matches Job 1003 with similarity 0.82
Resume 5 best matches Job 1108 with similarity 0.68


In [ ]:
def match_resume_to_job(resume_text):
    resume_embedding = model.encode([resume_text], convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(resume_embedding, job_embeddings).numpy()
    top_match = np.argmax(similarities)

    return f"Best matched job: {job_df.iloc[top_match]['Job_Title']}"

# Gradio UI
gr.Interface(fn=match_resume_to_job, inputs="textbox", outputs="text").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9bb9e5f5fe4037789d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/match_resume/")
def match_resume(resume_text: str):
    resume_embedding = model.encode([resume_text], convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(resume_embedding, job_embeddings).numpy()
    top_match = np.argmax(similarities)

    return {"Best Matched Job": job_df.iloc[top_match]['Job_Title']}

In [ ]:
from google.colab import files
import pandas as pd

# Assuming 'top_matches' from previous cell contains the results
# Create a DataFrame with the matched results
matched_results_df = pd.DataFrame({'Resume_Index': range(1, len(top_matches) + 1),
                                     'Matched_Job_Index': top_matches + 1})  # +1 to adjust index starting from 1

# Save the DataFrame to a CSV file
matched_results_df.to_csv("matched_results.csv", index=False)

# Now download the file
files.download("matched_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>